## Language Detector Project

Dataset Source: https://www.kaggle.com/datasets/basilb2s/language-detection

##### Import Necessary Libraries

In [0]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, DoubleType, FloatType, ArrayType

from pyspark.ml import Pipeline

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

##### Versions of Libraries, Modules, Frameworks Used in This Project

In [0]:
print("Apache Spark version:", spark.version)
print("Spark NLP version:", sparknlp.version())

Apache Spark version: 3.2.1
Spark NLP version: 4.2.5


#### Create Functions Used Throughout This Project

##### Create Function to Ingest Dataset

In [0]:
def ingest_dataset(file_location: str, \
                   schema: StructType \
                  ) -> pyspark.sql.dataframe.DataFrame:
    '''
    This function returns a dataset aligned with the schema
    '''
    file_type = "csv"
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ","
    
    df = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .schema(schema)\
      .load(file_location)
    
    return df

##### Create Metrics Evaluation Function

In [0]:
def evaluate_with_spark_metrics(predictions: pyspark.sql.dataframe.DataFrame, \
                                metrics: [str], \
                                model_name: str \
                               ) -> None:
    '''
    Calculate & display metrics for a multiclass classification analysis.
    '''
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    print("+---------------------------------------------+")
    print("|  " + model_name.center(41) + "  |")
    print("+---------------------------------------------+")
    print("|   %s  |  %s   |" % ("Metric".rjust(20), "Value".ljust(14)))
    print("+---------------------------------------------+")
    for x in metrics:
        evaluator = MulticlassClassificationEvaluator(labelCol="label", \
                                                      predictionCol="prediction", \
                                                      metricName=x) 
        score = evaluator.evaluate(predictions)
        print("|   %s  |  %s   |" % (x.rjust(20), str(round(score, 6)).ljust(14)))
        print("+---------------------------------------------+")

#### Ingest & Preprocess Datasets

##### Ingest Dataset

In [0]:
data_file = "/FileStore/tables/Language_Detection.csv"

orig_schema = StructType([
    StructField("text", StringType(), True),
    StructField("language", StringType(), True),
])

df = ingest_dataset(data_file, orig_schema)

display(df)

text,language
"Nature, in the broadest sense, is the natural, physical, material world or universe.",English
"""""""Nature"""" can refer to the phenomena of the physical world","and also to life in general."""
"The study of nature is a large, if not the only, part of science.",English
"Although humans are part of nature, human activity is often understood as a separate category from other natural phenomena.",English
"""[1] The word nature is borrowed from the Old French nature and is derived from the Latin word natura, or """"essential qualities","innate disposition"""""
"[2] In ancient philosophy, natura is mostly used as the Latin translation of the Greek word physis (φύσις), which originally related to the intrinsic characteristics that plants, animals, and other features of the world develop of their own accord.",English
[3][4],null
The concept of nature as a whole,the physical universe
"During the advent of modern scientific method in the last several centuries, nature became the passive reality, organized and moved by divine laws.",English
"[5][6] With the Industrial revolution, nature increasingly became seen as the part of reality deprived from intentional intervention : it was hence considered as sacred by some traditions (Rousseau, American transcendentalism) or a mere decorum for divine providence or human history (Hegel, Marx).",English


##### Data Preprocessing (Part 1)

In [0]:
# Remove duplicates
df = df.dropDuplicates()

# Remove null values
df = df.na.drop(how='any')

df = df.withColumnRenamed("language", "label")

# Add feature with word length of 'text' &  'language' features (individually)
df = df.withColumn("text_len", F.size(F.split(F.col("text"), " ")))
df = df.withColumn("language_len", F.size(F.split(F.col("label"), " ")))

# Remove samples with language feature that is more than one word
df = df.filter(F.col("language_len") < 2)

languages_for_analysis = ["English", "Italian", "Spanish", "French", "Dutch"]

df = df.filter(F.col("label").isin(languages_for_analysis))

display(df)

text,label,text_len,language_len
"The number of active editors in English Wikipedia, by sharp comparison, was cited as peaking in 2007 at approximately 50,000 and dropping to 30,000 by the start of 2014.",English,29,1
[143] No comment was made concerning which of the differentiated edit policy standards from Wikipedia in other languages (non-English Wikipedia) would provide a possible alternative to English Wikipedia for effectively ameliorating substantial editor attrition rates on the English-language Wikipedia.,English,39,1
"The 'Wikipedia is not censored' policy has sometimes proved controversial: in 2008, Wikipedia rejected an online petition against the inclusion of images of Muhammad in the English edition of its Muhammad article, citing this policy.",English,35,1
"This pattern does not adhere to the common statistical definition of an outlier as a rare object, and many outlier detection methods (in particular, unsupervised algorithms) will fail on such data unless it has been aggregated appropriately.",English,37,1
"The present era is classified as part of a mass extinction event, the Holocene extinction event, the fastest ever to have occurred.",English,22,1
[64] These life forms possessed the basic traits of self-replication and inheritable traits.,English,13,1
"[55][56] In June 2019, scientists reported that all 16 GB of article text from the English Wikipedia have been encoded into synthetic DNA.",English,22,1
"[68] Under this system, new and unregistered users' edits to certain controversial or vandalism-prone articles are reviewed by established users before they are published.",English,24,1
"Content in Wikipedia is subject to the laws (in particular, copyright laws) of the United States and of the US state of Virginia, where the majority of Wikipedia's servers are located.",English,31,1
"[137] Though each language edition functions more or less independently, some efforts are made to supervise them all.",English,18,1


##### Data Preprocessing (Part 2)

In [0]:
df = df.drop("language_len", "text_len")

df = df.persist()

##### Build Pipeline Stages

In [0]:
# Document Assembler
doc = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document") \
    .setCleanupMode('shrink')

# Language Detector
language_detector = LanguageDetectorDL.pretrained() \
    .setInputCols("document") \
    .setOutputCol("class")

ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
[OK!]


##### Build Pipeline

In [0]:
pipeline = Pipeline().setStages([
      doc,
      language_detector
])

##### Generate Predictions

In [0]:
preds = pipeline.fit(df).transform(df)

preds = preds.select(F.col("label"), F.col("class.result").alias("prediction"))
preds = preds.withColumn("prediction", F.col("prediction").getItem(0))
display(preds)

label,prediction
English,en
English,en
English,en
English,en
English,en
French,fr
French,en
French,fr
French,fr
French,fr


##### Prepare Predictions for Metrics Evaluation Function

In [0]:
languages_converter = {"English" : "0", "Italian" : "1", "Spanish" : "2", "French" : "3", "Dutch" : "4"}
preds = preds.replace(languages_converter, subset=["label"]).withColumn("label", F.col("label").cast(DoubleType()))

preds_converter = {"en" : "0", "it" : "1", "es" : "2", "fr" : "3", "nl" : "4"}
preds = preds.replace(preds_converter, subset=["prediction"]).withColumn("prediction", F.col("prediction").cast(DoubleType()))

preds = preds.fillna(-1)

preds = preds.persist()

display(preds)

label,prediction
0.0,0.0
0.0,0.0
0.0,0.0
0.0,0.0
0.0,0.0
3.0,3.0
3.0,0.0
3.0,3.0
3.0,3.0
3.0,3.0


##### Calculate & Display Metrics

In [0]:
metrics_to_eval = ["accuracy", "f1", "weightedPrecision", "weightedRecall"]

evaluate_with_spark_metrics(preds, \
                            metrics_to_eval, \
                            "Metrics for Cancer Classification")

+---------------------------------------------+
|      Metrics for Cancer Classification      |
+---------------------------------------------+
|                 Metric  |  Value            |
+---------------------------------------------+
|               accuracy  |  0.989564         |
+---------------------------------------------+
|                     f1  |  0.992089         |
+---------------------------------------------+
|      weightedPrecision  |  0.994661         |
+---------------------------------------------+
|         weightedRecall  |  0.989564         |
+---------------------------------------------+


##### End Spark Session

In [0]:
df = df.unpersist()
preds = preds.unpersist()

spark.stop()

### Notes & Other Takeaways From This Project
****
- The results were about what I expected! Accuracy, F1, recall, and precision near or above 0.99.
****
- I filled all predictions that were 'null' with -1 to demonstrate that the model did not correctly predict their value.
****